In [2]:
import sys
sys.path.append("..")
import pandas as pd
import numpy as np
import pickle
import tqdm
import pymorphy2
import logging
import os
from string import punctuation
from nltk import TreebankWordTokenizer
from nltk.corpus import stopwords
from sklearn.metrics.pairwise import cosine_similarity
from tqdm.auto import tqdm
from sqlalchemy import create_engine
from src.config import conn_string

In [3]:
morph = pymorphy2.MorphAnalyzer()
logging.basicConfig(level="INFO")

In [4]:
def get_lines(conn_string):
    """
    Подключается к БД и выкачивает вакансии
    """
    logging.info("Подгружаю данные из базы")
    engine = create_engine(conn_string)

    df = pd.read_sql_table('vacancy', engine)
    logging.info(df.head)
    lines = df.vacdescription.tolist()
    vacids = df.vacid.tolist()
    return lines, vacids

In [5]:
def txt_pipe(lines):
    logging.info("Готовлю корпус")
    ru_stop_words = stopwords.words('russian')
    lines_tok = [TreebankWordTokenizer().tokenize(x) for x in lines]
    lines_tok = [[x for x in el if x not in punctuation] for el in lines_tok]
    u_norm = [[morph.parse(x)[0][2] for x in el] for el in tqdm(lines_tok)]
    u_norm = [[x for x in el if x not in ru_stop_words] for el in tqdm(u_norm)]
    corpus = [' '.join(x) for x in u_norm]
    return corpus

In [6]:

def l2_norm(x):
    return np.sqrt(np.sum(x ** 2))


def div_norm(x):
    norm_value = l2_norm(x)
    if norm_value > 0:
        return x * (1.0 / norm_value)
    else:
        return x

In [7]:
def get_vacancy_vectors(vacids, corpus):
    """
    Получает вектора профилей пользователей из фасттекста
    """

    from gensim.models import FastText
    
    vacancy_vectors = {}
    logging.info("Подгружаем обученную модель FastText")
    fasttext_pth = os.path.join('..','wvmodel','cc.ru.300.bin')
    fast_text = FastText.load_fasttext_format(fasttext_pth).wv
    
    logging.info("Собираем векторы предложений")
    for x in tqdm((vacids, corpus)):

        text = x[1].split()
        text.append('\n')
        matrix = np.zeros((300,), dtype = 'float32')

        for word in text:
            matrix += div_norm(fast_text.word_vec(word))

        vacancy_vectors[x[0]] = matrix

In [8]:

def get_similarities(target_user, candidates):
    """Получает косинусные сходства сотрудника и кандидатов"""

    tu_sims = {}
    for vacid in vacids:
        tu_sims[candidate] = cosine_similarity(
            vacancy_vectors[vacid],
            user_vectors
        )[0][0]

    return tu_sims

In [9]:
lines, vacids = get_lines(conn_string)

INFO:root:Подгружаю данные из базы
INFO:root:<bound method NDFrame.head of           id                 created_at                 updated_at   vacid  \
0     156714 2020-12-17 20:17:18.625748 2020-12-17 20:17:18.625881   78349   
1     156715 2020-12-17 20:17:18.625748 2020-12-17 20:17:18.625881   99119   
2     156716 2020-12-17 20:17:18.625748 2020-12-17 20:17:18.625881   99132   
3     156717 2020-12-17 20:17:18.625748 2020-12-17 20:17:18.625881  368975   
4     156718 2020-12-17 20:17:18.625748 2020-12-17 20:17:18.625881  370412   
...      ...                        ...                        ...     ...   
3402  160116 2020-12-17 20:17:18.625748 2020-12-17 20:17:18.625881  453073   
3403  160117 2020-12-17 20:17:18.625748 2020-12-17 20:17:18.625881  453796   
3404  160118 2020-12-17 20:17:18.625748 2020-12-17 20:17:18.625881  461680   
3405  160119 2020-12-17 20:17:18.625748 2020-12-17 20:17:18.625881  476181   
3406  160120 2020-12-17 20:17:18.625748 2020-12-17 20:17:18.625881 

In [10]:
corpus = txt_pipe(lines)

INFO:root:Готовлю корпус


In [11]:
# with open(os.path.join('..','data','corpus.pkl'), 'wb') as f:
#     pickle.dump(corpus, f)

# with open(os.path.join('..','data','vacids.pkl'), 'wb') as f:
#     pickle.dump(vacids, f)

In [6]:
with open(os.path.join('..','data','vacids.pkl'), 'rb') as f:
    vacids = pickle.load(f)

with open(os.path.join('..','data','corpus.pkl'), 'rb') as f:
    corpus = pickle.load(f)

In [29]:
vvect = get_vacancy_vectors(vacids, corpus)

<ipython-input-24-dae2beaa0cb4>:11: DeprecationWarning: Call to deprecated `load_fasttext_format` (use load_facebook_vectors (to use pretrained embeddings) or load_facebook_model (to continue training with the loaded full model, more RAM) instead).
  fast_text = FastText.load_fasttext_format(fasttext_pth).wv


In [9]:
# logging.info("Подгружаем обученную модель FastText")

In [3]:
from gensim.models import fasttext
# fasttext_pth = os.path.join('..','wvmodel','cc.ru.300.bin')
# fast_text = fasttext.load_facebook_vectors(fasttext_pth)

with open(os.path.join('..','data','fast_text.pkl'), 'rb') as f:
    fast_text = pickle.load(f)

In [14]:
# with open(os.path.join('..','data','fast_text.pkl'), 'wb') as f:
#     pickle.dump(fast_text, f)

In [13]:
vacancy_vectors = {}
for x in tqdm(list(zip(vacids, corpus))):
    text = x[1].split()
    text.append('\n')
    matrix = np.zeros((300,), dtype = 'float32')
    for word in text:
        matrix += div_norm(fast_text.word_vec(word))
    vacancy_vectors[x[0]] = matrix

In [70]:
text = "python git data science machine learning"
text = "сми репутация сторителлинг фактчекинг пресс-релиз коммуникация pr журналист москва"
text = text.split()
text.append('\n')
matrix = np.zeros((300,), dtype = 'float32')
for word in text:
    matrix += div_norm(fast_text.word_vec(word))

In [71]:
tu_sims = {}
for vacid in tqdm(vacids):
    tu_sims[vacid] = cosine_similarity(vacancy_vectors[vacid].reshape(1,-1),
                                       matrix.reshape(1,-1))[0][0]

In [75]:
tu_sorted = sorted(tu_sims.items(), key=lambda x:x[1], reverse=True)
tu_sorted = [x[0] for x in tu_sorted]
df = pd.DataFrame({'description':lines, 'vacid':vacids})
df = df.set_index('vacid')

In [83]:
df.loc[tu_sorted[]].description

' Условия:Профессиональное обучение, семинары, тренинги, конференции;Квартальные и годовые премии;ДМС, сниженные ставки по кредитованию, программы лояльности для сотрудников;Самые инновационные, амбициозные проекты и задачи.Обязанности:Написание самостоятельных текстов/статей: лонгриды, заметки, подводки, тезисы и т. д.;Написание и редактирование текстов для новостей, статей, пресс-релизов, презентаций, соцсетей, рассылок и интерфейсов;Проработка контент-планов;Коммуникации с проджектами и дизайнерами (подготовка ТЗ и брифов);Участие в мероприятиях, подготовка речи/выступлений, оперативная подготовка репортажей;Вычитка и редактирование текстов в электронном виде с целью обеспечения графического и лексического единообразия различных элементов текста, устранения орфографических и пунктуационных ошибок, а также исправления недостатков смыслового и стилистического характера;Редактура текстов других авторов, рерайтинг. Делать из слабого текста сильный: менять структуру, корректировать стили